# ILP: Практическое задание

В этом ноутбуке 6 задач:
- В первом разделе ты найдешь четыре задачи на принятие решений, которые тебе нужно решить с помощью CVXPY,
- Во втором разделе ты найдешь две задачи иссследовательского типа;

**Самопроверка** Для задач первого раздела мы написали один тест, на котором ты можешь проверить свое решение. Перед отправкой своего решения убедись, что твой код дает верный ответ.

*Замечание* Прохождение теста не гарантирует, что твое решение будет зачтено. Мы дополнительно проверим, что твой код написан без ошибок.

In [ ]:
# Импортируем библиотеку
import cvxpy as cp
import numpy as np

# **Раздел 1**

# Задача 1. Рюкзак

<img src='https://drive.google.com/uc?id=1UsNQLwcLuYUABb_KG7YtNMfmV2Lhvq8i
' width='800'>

In [ ]:
# задаем параметры
W = 15
values  =  np.array([10, 20, 9, 25, 15])
weights =  np.array([3,  5,  2,  7, 4])

In [ ]:
# РЕШЕНИЕ ЗАДАЧИ 1

def solve_knapsack(values, weights, W):

  """
  Функция для решения задачи рюкзака.

  Args:
      параметры задачи

  Returns:
      + оптимальное значение целевой функции
      + оптимальное значение переменных
  """

  x = cp.Variable(len(weights), boolean=True)

  func = values @ x
  constraints = [weights @ x <= W]

  optimal_value = cp.Problem(cp.Maximize(func),constraints)
  optimal_value = optimal_value.solve()
  optimal_variables = x.value

  return optimal_value, optimal_variables

In [ ]:
# ТЕСТ. Если ты верно решил(а) задачу, это ячейка должна выполниться без ошибок
# Не меняй код в этой ячейке

optimal_value, optimal_variables = solve_knapsack(values, weights, W)

assert optimal_value == 55.0

# Задача 2. Склады



<img src='https://drive.google.com/uc?id=1yY7PaR3JPaTUwqDHXZz6L90AiwTS6Wj1
' width='600'>

# Формулировка

**Переменные**

1. Для каждой потенциальной локации $i$ вводим бинарную переменную $x_i \in \{0, 1\}$. Переменная $x_i$ равна единице, если мы арендуем склад в локации $i$.

2. Для каждой пары (локация, клиент) введем переменную $y_{ij}$: количество партий воды, которые доставляются клиенту $j$ со склада $i$.

---


**Целевая функция**

Комания «X» заинтересована в минимизации суммарных недельных затрат. С помощью введенных переменных суммарные затраты можно записать так:

$$\sum\limits_i \texttt{fixed_cost}_i \cdot x_i + \sum\limits_{(i, j)} \texttt{variable_cost}_{ij} \cdot y_{ij}.$$

- Первое слагаемое отражает суммарную стоимость недельной аренды открытых складов.

- Второе слагаемое отражает стоимость, которую компании нужно тратить на доставку воды до клиентов.

---

**Ограничения**

Введем ограничения, чтобы отразить суть задачи.

1. Потребность каждого клиента должна быть удовлетворена: $$\forall j \quad \sum\limits_i y_{ij} = \texttt{demand}_j.$$

2. Со склада нельзя отправить больше воды, чем его вместимость. При этом если компания не взяла склад в аренду, то его вместимость равна 0: $$\forall i \quad \sum\limits_j y_{ij} \le \texttt{capacity}_i \times x_i.$$

3. Допустимые значения переменных: $$x_i \in \{0, 1\}, \ y \in \mathbb{Z}^{+}.$$



In [ ]:
# задаем параметры

#количество локаций и клиентов
num_locations = 3
num_customers = 4

#стоимость недельной аренды складов
fixed_costs = np.array([500, 700, 300])

#вместимость складов
capacities = np.array([100, 80, 120])

#потребность клиентов
demands = np.array([50, 60, 40, 70])

#стоимость доставки
variable_costs = np.array([
    [2, 3, 1, 4],
    [3, 1, 2, 3],
    [4, 2, 5, 1]
])

In [ ]:
# РЕШЕНИЕ ЗАДАЧИ 2

def solve_location(num_locations, num_customers, fixed_costs,
                   capacities, demands, variable_costs):

  """
  Функция для решения задачи определения локаций.

  Args:
      параметры задачи

  Returns:
      + оптимальное значение целевой функции
      + оптимальное значение переменных
  """

  #твой код здесь
  x = cp.Variable(num_locations,boolean=True)
  y = cp.Variable((num_locations,num_customers),integer=True)

  value = fixed_costs @ x + cp.sum(cp.multiply(variable_costs,y))

  constraints = []
  constraints.append(cp.sum(y,axis=0) == demands)
  constraints.append(cp.sum(y,axis=1) <= cp.multiply(capacities, x))
  constraints.append(y >= 0)

  optimal_value = cp.Problem(cp.Minimize(value),constraints)
  optimal_value = optimal_value.solve()
  optimal_variables = [x.value, y.value]

  return optimal_value, optimal_variables

In [ ]:
# ТЕСТ. Если ты верно решил(а) задачу, это ячейка должна выполниться без ошибок
# Не меняй код в этой ячейке

optimal_value, optimal_variables = solve_location(num_locations, num_customers,
                                                  fixed_costs, capacities,
                                                  demands, variable_costs)


assert optimal_value == 1140.0

# Задача 3. Оптимизация представителей I


<img src='https://drive.google.com/uc?id=1pUUArxSgxd1VGT01c2X2ox_NQAlsQG_z
' width='1400'>

In [ ]:
# задаем параметры

#спрос по дням недели: сколько представителей должны работать в каждый из дней
demand = np.array([11, 14, 13, 15, 12, 8, 7])

#рейтинг представителей: чем выше рейтинг, тем лучше.
#размер списка совпадает с общим количеством представителей
employee_rating = np.array([99, 96,  5, 92,  1, 46,  6, 27, 74, 18, 30,  7, 75, 12, 68,  9, 22,
       96, 41, 56,  6, 95, 89, 42, 41, 49, 58, 97, 73, 31])

#дни, в которые представители недоступны
#key: индекс представителя
#value: дни, в которые данный представитель недоступен
unavailable_days = {0: np.array([6]),
 1: np.array([0, 1, 6]),
 2: np.array([0, 1, 4, 5, 6]),
 3: np.array([3, 6]),
 4: np.array([]),
 5: np.array([0, 3, 5, 6]),
 6: np.array([2, 4]),
 7: np.array([0, 1, 2]),
 8: np.array([3]),
 9: np.array([0, 1, 4, 6]),
 10: np.array([0]),
 11: np.array([2, 4]),
 12: np.array([]),
 13: np.array([0]),
 14: np.array([3]),
 15: np.array([1]),
 16: np.array([0, 1, 3, 4, 5]),
 17: np.array([]),
 18: np.array([0, 1, 2]),
 19: np.array([1]),
 20: np.array([1, 2, 3, 4, 6]),
 21: np.array([1, 3, 4, 6]),
 22: np.array([4]),
 23: np.array([2, 5]),
 24: np.array([2, 3, 6]),
 25: np.array([]),
 26: np.array([6]),
 27: np.array([1, 6]),
 28: np.array([0, 5]),
 29: np.array([])}

In [ ]:
# РЕШЕНИЕ ЗАДАЧИ 3

def get_optimal_schedule(demand, employee_rating, unavailable_days):

    """
    Функция для составления расписания иполнителей.

    Args:
        параметры задачи

    Returns:
        + оптимальное значение целевой функции
        + оптимальное значение переменных
    """

    #твой код здесь
    schedule = cp.Variable((len(employee_rating),len(demand)), boolean=True)
    constraints = []

    for emp in range(len(employee_rating)):
      for day in unavailable_days.get(emp,[]):
        constraints.append(schedule[emp,day] == 0)
    for day in range(len(demand)):
      constraints.append(cp.sum(schedule[:,day]) == demand[day])

    rat = cp.sum(cp.multiply(employee_rating[:, None], schedule))

    optimal_value = cp.Problem(cp.Maximize(rat),constraints)
    optimal_value = optimal_value.solve()
    optimal_variables = schedule.value

    return optimal_value, optimal_variables


In [ ]:
# ТЕСТ. Если ты верно решил(а) задачу, это ячейка должна выполниться без ошибок
# Не меняй код в этой ячейке


optimal_value, optimal_variables = get_optimal_schedule(demand, employee_rating,
                                                        unavailable_days)

assert optimal_value == 6062.0

# Задача 4. Оптимизация представителей II

Усложним задачу:
- теперь у тебя есть два региона (0 и 1)
- представители живут в одном из двух регионов
- работа представителя в *чужом регионе* один день эквивалентна потери λ единиц суммарного рейтинга
- в один день представитель может работать только в одном регионе
- в некоторые дни представителей может не хватать на покрытие всего спроса

Тебе нужно составить такое расписание, которое максимизирует целевую функцию.

**Целевая функция в этой задаче** Сумма рейтингов работающих преставителей — штраф за работу в чужом регионе


In [ ]:
# задаем параметры

#спрос по дням недели: сколько представителей должны работать в каждый из дней в каждом регионе
demand_1 = [35, 33, 30, 27, 26, 17, 15]
demand_2 = [20, 17, 15, 19, 18, 13, 10]

#рейтинг представителей: чем выше рейтинг, тем лучше.
#размер списка совпадает с общим количеством представителей
employee_rating = np.array([83, 66, 30, 64, 80, 29, 71, 50, 40, 16,  6, 30, 36, 93, 72, 21,  9,
                            48, 72, 51, 78, 22, 31,  7, 66, 36, 11, 10, 82, 50, 58, 31, 13, 65,
                            48, 35, 10, 14, 81, 91, 41, 80, 28, 39, 61, 39, 97, 85, 48])

# регионы, в которых живут представители
employee_home_regions = np.array([0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
                                  0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
                                  0, 0, 0, 0, 0])

# штраф за работу представителя в чужом регионе
lambda_param = 50

#дни, в которые представители недоступны
#key: индекс представителя
#value: дни, в которые данный представитель недоступен
unavailable_days = {0:np.array([]),
                    1:np.array([0, 1, 2, 4, 6]),
                    2:np.array([0, 2, 4, 6]),
                    3:np.array([4, 5]),
                    4:np.array([0]),
                    5:np.array([0, 1, 3, 4, 6]),
                    6:np.array([0, 1, 5, 6]),
                    7:np.array([5, 6]),
                    8:np.array([0, 2]),
                    9:np.array([0, 1, 5, 6]),
                    10:np.array([5,6]),
                    11:np.array([0]),
                    12:np.array([]),
                    13:np.array([0, 2, 3, 5]),
                    14:np.array([]),
                    15:np.array([4, 5]),
                    16:np.array([0, 4, 5]),
                    17:np.array([4]),
                    18:np.array([]),
                    19:np.array([0, 1, 2, 5]),
                    20:np.array([0, 2, 3, 4]),
                    21:np.array([2]),
                    22:np.array([0, 3, 6]),
                    23:np.array([]),
                    24:np.array([]),
                    25:np.array([2]),
                    26:np.array([1, 2, 3, 5, 6]),
                    27:np.array([1, 3, 4, 5]),
                    28:np.array([0]),
                    29:np.array([4, 5]),
                    30:np.array([0, 1, 3, 4, 6]),
                    31:np.array([0, 1, 2, 6]),
                    32:np.array([]),
                    33:np.array([1, 4, 5]),
                    34:np.array([0, 2, 3, 4, 5]),
                    35:np.array([4]),
                    36:np.array([1, 2, 3, 5]),
                    37:np.array([0, 1, 4, 6]),
                    38:np.array([0, 1, 6]),
                    39:np.array([1, 2, 4, 6]),
                    40:np.array([0, 4, 5]),
                    41:np.array([0, 2, 4, 6]),
                    42:np.array([3, 4, 5]),
                    43:np.array([0, 5, 6]),
                    44:np.array([2]),
                    45:np.array([0, 1, 2, 3]),
                    46:np.array([2, 4, 5]),
                    47:np.array([5,6]),
                    48:np.array([1, 2])
                    }

In [ ]:
#РЕШЕНИЕ ЗАДАЧИ 4

def get_optimal_schedule2(demand_1, demand_2, employee_home_regions,
                         employee_rating, unavailable_days, lambda_param):

    """
    Функция для составления расписания иполнителей для нескольких регионов.

    Args:
        параметры задачи

    Returns:
        + оптимальное значение целевой функции
        + оптимальное значение переменных
    """

    #твой код здесь
    schedule1 = cp.Variable((len(employee_rating),len(demand_1)),boolean=True)
    schedule2 = cp.Variable((len(employee_rating),len(demand_2)),boolean=True)
    schs = schedule1 + schedule2

    constraints = []

    for per in range(len(employee_rating)):
      for day in unavailable_days.get(per,[]):
        constraints.append(schedule1[per,day] == 0)
        constraints.append(schedule2[per,day] == 0)

    for per in range(len(employee_rating)):
      for day in range(len(demand_1)):
        constraints.append((schedule1[per,day] + schedule2[per,day]) <= 1)


    for day in range(len(demand_1)):
      constraints.append(cp.sum(schedule1[:, day]) <= demand_1[day])
      constraints.append(cp.sum(schedule2[:,day]) <= demand_2[day])


    forfeit = cp.sum(lambda_param * (cp.multiply((employee_home_regions == 0)[:, None], schedule2) + cp.multiply((employee_home_regions == 1)[:, None], schedule1)))

    rat = cp.sum(cp.multiply(employee_rating[:, None],schs)) - forfeit
    optimal_value = cp.Problem(cp.Maximize(rat),constraints)
    optimal_value = optimal_value.solve()
    optimal_variables = {'schedule1': schedule1.value, 'schedule2':schedule2.value}

    return optimal_value, optimal_variables

In [ ]:
# ТЕСТ #1. Если ты верно решил(а) задачу, это ячейка должна выполниться без ошибок
# Не меняй код в этой ячейке

optimal_value, optimal_variables = get_optimal_schedule2(demand_1, demand_2,
                                                        employee_home_regions,
                                                        employee_rating,
                                                        unavailable_days,
                                                        lambda_param=1)

assert optimal_value == 10361.0

In [ ]:
# ТЕСТ #2. Если ты верно решил(а) задачу, это ячейка должна выполниться без ошибок
# Не меняй код в этой ячейке

optimal_value, optimal_variables = get_optimal_schedule2(demand_1, demand_2,
                                                        employee_home_regions,
                                                        employee_rating,
                                                        unavailable_days,
                                                        lambda_param=50)

assert optimal_value == 10307.0

# **Раздел 2**

# Задача 5. Исследование решений

Возьми задачу по производству столов и стульев из тьюториала и проверь, как меняется оптимальное решение (x1 и x2) при изменении цены стула и цены стола (коэффициенты целевой функции).

**Диапазон изменения цен** Рассмотри все возможные комбинации цен на столы и стулья в диапазоне от 100 до 1000. Считай, что цены на каждый предмет должны быть кратны 100.

**Анализ** Какое количество разных решений ты получил(а)? Есть ли что-то особенное у этих значений?


In [ ]:
from itertools import product

In [ ]:
prices = np.array([100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])

p1_pr = prices
p2_pr = prices

results = {}

In [ ]:
for p1, p2 in product(p1_pr,p2_pr):
  x1 = cp.Variable(integer=True)
  x2 = cp.Variable(integer=True)

  constraints = []
  constraints.append(x1 + 2*x2 <= 6000)
  constraints.append(3*x1 + x2 <= 9000)
  constraints.append(x1 >= 0)
  constraints.append(x2 >= 0)

  rat = p1*x1 + p2*x2
  res = cp.Problem(cp.Maximize(rat),constraints)
  res = res.solve()

  sol = (int(x1.value),int(x2.value))
  if sol in results:
    results[sol] += 1
  else:
    results[sol] = 1

In [ ]:
for results,count in sorted(results.items()):
  print(results, count)

(0, 3000) 25
(2400, 1800) 60
(3000, 0) 15


- всего три различных решения
- цены влияют на выбор между производством столов или стульев, но комбинации не дают новых решений из-за ограничения по ресурсам

# Задача 6. Сравнение подходов к распределению статей

На лекции мы рассмотрели два подхода к распределению статей по рецензентам

<img src='https://drive.google.com/uc?id=1TaeTNjtscSz5J5iYX-X-lnx6KZzLQaCh
' width='800'>

В этой задаче тебе нужно будет сравнить скорость решения задачи оптимизации для двух подходов.

**Параметры задачи** Будем считать, что:
- количество статей равно количеству рецензентов ($m = n$)
- каждую статью должны проверить три рецензента
- каждый рецензент может проверить до четырех статей
- конфликтов интересов нет

**Твоя задача** Для разных значений $n$ сгенерируй матрицу similarity $S$ и измерь время, которое cvxpy тратит на поиск оптимального решения в каждом из подходов. В качестве решения приведи график, на котором отложено время решения в зависимости от $n$ для каждого из подходов. Прокомментируй результат.

**Подсказки**
- начни с $n=5$ и увеличивай до тех пор, пока задача решается за разумное время
- для каждого $n$ тебе может быть полезно решить задачу для нескольких $S$, чтобы усреднить результаты


In [ ]:
#твое решение здесь